companies dont have MKTCAP values have their MKTCAP valeus documented as NaN in DB, while 0 in Excel.

if the number of zero MKTCAP in Excel is more than the zeros in DB (and their tickers are not overlapping), which could mean that there actually are zero MKTCAP companies, apart from companies having no MKTCAP vlaues.

remove the companies without MKTCAP values in 2015, around 91 of them

do the same thing in 2016 

and merge the data from hte two years 

In [1]:
import pandas as pd
import numpy as np

In [41]:
df = pd.read_csv('MKTCAP_LN_UNADJ.csv', encoding = 'cp932')

# df.shape  # 16711 x 52
df = df.drop(df.columns[0], axis = 1 )
df.head()

,security_code,ddate,売上,営業利益,純利益,総資産,現金･現金同等物,有価証券,流動資産,総長期投資,...,長期社債(当期返済分),長期借入金(当期返済分),少数持分,自己株金額,人件費,税引前利益,非流動資産合計,自己株式数,発行済株式数,value
0,AAL,2000,14824.000,2368.000,1957.000,30616.000,1061.000,2344.000,8127.000,935.000,...,0,0.0,2646.0,-0.0,-3067.000,3444.000,22489.000,0.0,1483.8861,22469.2680
1,ABF,2000,4406.000,327.000,138.000,3867.000,65.000,1133.000,2220.000,4.000,...,0,0.0,78.0,-0.0,-581.000,245.000,1647.000,0.0,791.6740,2652.1079
2,ACL,2000,245.360,13.484,8.232,146.453,8.461,0.000,86.766,0.000,...,0,0.0,0.0,-0.0,-28.867,12.451,59.687,0.0,35.4745,116.6681
3,ADB,2000,32.645,2.448,1.466,23.528,0.412,0.000,6.764,0.677,...,0,0.0,0.0,-0.0,-5.897,2.239,16.764,0.0,0.4720,20.2960
4,ADN,2000,115.721,34.225,24.126,274.148,45.815,3.054,91.628,25.218,...,0,0.0,0.0,-0.0,-34.826,33.961,182.520,0.0,240.4588,884.2819


In [28]:
df[df['ddate'] == 2015]['value'].describe()

count      1362.000000
mean       2395.904387
std       17693.725001
min           0.000000
25%           7.833900
50%          51.008350
75%         375.677350
max      375211.984500
Name: value, dtype: float64

In [27]:
df[df['ddate'] == 2015]['security_code'].describe()

count     1362
unique    1362
top        TAX
freq         1
Name: security_code, dtype: object

In [40]:
# any NA for unadjusted MKTCAP 
df.isnull().any(axis = 0)['value']

False

# from DB

In [42]:
tk_2015 = pd.read_csv('tk_2015.csv', encoding = 'cp932')
tk_2015.head()

,security_code
0,7DIG
1,888
2,AA/
3,AAA
4,AAL


In [101]:
tk_mc2015 = pd.read_csv('tk_mc2015.csv', encoding = 'cp932')


tk_mc2015.head()

,security_code,value
0,7DIG,7.7257
1,888,960.1755
2,AA/,1981.9350
3,AAA,32.9880
4,AAL,5667.4586


In [68]:
# tk_mc2015 = pd.read_csv('tk_mc2015.csv', encoding = 'cp932')

# tk_mc2015['with_mktcap']  = tk_mc2015['security_code']

# del tk_mc2015['security_code']

# tk_mc2015.head()

In [82]:
outer = pd.merge(tk_2015, tk_mc2015, on = 'security_code', how = 'outer')

print (outer.shape)

outer.head()

(1365, 2)


,security_code,value
0,7DIG,7.7257
1,888,960.1755
2,AA/,1981.9350
3,AAA,32.9880
4,AAL,5667.4586


In [141]:
# these values are stored as NaN in the DB, while as 0.0 in Excel 

outerzero = outer[outer.isnull().any(axis = 1)]

print(outerzero.shape)
outerzero.head()

(91, 2)


,security_code,value
17,ACRL,NaN
24,ADES,NaN
36,AFID,NaN
71,ANGS,NaN
87,ARS,NaN


In [139]:
print (tk_2015.shape)
# which is the same as the result from querry in the unadjusted data set 
print (df[df['ddate'] == 2015].shape)

# tickers with market cap values 
print(tk_mc2015.shape)

tk_2015.shape[0] - tk_mc2015.shape[0]

(1362, 1)
(1362, 52)
(1274, 2)


88

In [140]:
# working on the unadjusted mktcap dataset from .csv - results are different for 2 entries

dataset = df[df['value'] == 0][df[df['value'] == 0]['ddate'] == 2015][['security_code', 'value']]

print(dataset.shape)
dataset.head()

(91, 2)


,security_code,value
13763,ACRL,0.0
13770,ADES,0.0
13781,AFID,0.0
13816,ANGS,0.0
13832,ARS,0.0


In [131]:
set(list(dataset['security_code'].values)) - set(list(outerzero['security_code'].values))

set()

In [138]:
a = list(dataset['security_code'].values)
b = list(outerzero['security_code'].values)

print (len(a), len(b))

[x for x in a if x not in b ]

91 91


[]

In [81]:
df[df['security_code'] == 'WEY']

,security_code,ddate,売上,営業利益,純利益,総資産,現金･現金同等物,有価証券,流動資産,総長期投資,...,長期社債(当期返済分),長期借入金(当期返済分),少数持分,自己株金額,人件費,税引前利益,非流動資産合計,自己株式数,発行済株式数,value
9799,WEY,2011,0.1213,-0.3887,-0.3885,0.4540,0.3184,0.0,0.3687,0.0,...,0,0.0,0.0,-0.0,-0.0000,-0.3885,0.0852,0.0,11.5638,0.0000
11059,WEY,2012,0.2402,-0.3793,-0.4557,0.1332,0.0429,0.0,0.1281,0.0,...,0,0.0,0.0,-0.0,-0.0938,-0.4557,0.0051,0.0,13.8652,0.0000
12365,WEY,2013,0.1301,-0.1618,-0.1688,0.1251,0.0843,0.0,0.1235,0.0,...,0,0.0,0.0,-0.0,-0.0946,-0.1688,0.0015,0.0,13.8652,0.0000
13703,WEY,2014,0.0553,-0.2973,-0.3131,0.0529,0.0058,0.0,0.0529,0.0,...,0,0.0,0.0,-0.0,-0.1065,-0.3129,0.0000,0.0,13.8652,0.0000
15064,WEY,2015,0.0000,0.0000,0.0000,1.2995,0.0974,0.0,0.3168,0.0,...,0,0.0,0.0,-0.0,-0.0000,0.0000,0.9827,0.0,43.9711,0.0000
16411,WEY,2016,1.5158,-0.8023,-0.8258,1.9871,0.9099,0.0,1.1271,0.0,...,0,0.0,0.0,-0.0,-0.0000,-0.8066,0.8600,0.0,95.7712,3.4717


In [144]:
tk_2016 = pd.read_csv('tk_2016.csv', encoding = 'cp932')
tk_mc2016 = pd.read_csv('tk_mc2016.csv', encoding = 'cp932')

print (tk_2016.shape, tk_mc2016.shape)

tk_2016.shape[0] - tk_mc2016.shape[0]

(1347, 1) (1311, 2)


36

In [146]:
# based on data in the  DB

out = pd.merge(tk_2016, tk_mc2016, how = 'outer', on = 'security_code')

print(out.shape)
out.head()

(1354, 2)


,security_code,value
0,7DIG,7.3014
1,888,960.6034
2,AA/,1790.4874
3,AAA,20.2185
4,AAL,18308.3621


In [150]:
# companies without MKTCAP entries from DB

outcomp = out[out.isnull().any(axis = 1)]
print (outcomp.shape)
outcomp.head()

(43, 2)


,security_code,value
5,AAOG,NaN
17,ACRL,NaN
24,ADES,NaN
35,AFID,NaN
68,ANGS,NaN


In [154]:
# based on data from the unadjusted MKTCAP .csv

undata = df[df['value'] == 0][df[df['value'] == 0]['ddate'] == 2016][['security_code', 'value']]

print (undata.shape)
undata.head()

(43, 2)


,security_code,value
15113,AAOG,0.0
15125,ACRL,0.0
15132,ADES,0.0
15142,AFID,0.0
15175,ANGS,0.0


In [155]:
set(outcomp['security_code']) - set(undata['security_code'])

set()